Uploading frames with masks on the traffic signs

In [ ]:
!unzip /content/drive/Shareddrives/shared_Maor_Kfir/stop_test2/frame_with_masks.zip -d /content/extracted_folder

Creating a compressed video

In [ ]:
!ffmpeg -pattern_type glob -i '/content/extracted_folder/*.png' -pix_fmt yuv420p -s 1920x1080 out_1920_1080p.yuv

Uploading folders for kvazaar

In [ ]:
!git clone https://github.com/ultravideo/kvazaar.git

In [ ]:
cd /content/kvazaar

In [ ]:
!sudo apt install automake autoconf libtool m4 build-essential yasm


In [ ]:
! ./autogen.sh

In [ ]:
!./configure

In [ ]:
!make

In [ ]:
!sudo make install

In [ ]:
!sudo ldconfig

In [ ]:
cd ..

Change the format of the video from yuv to hevc for raising the quality




In [ ]:
!kvazaar -i out_1920_1080p.yuv --input-res 1920x1080 -o out_1000000.hevc --input-fps 30 --bitrate 1000000 --gop lp

Creating folders

In [ ]:
import os
# folder for black_images(the darkened image besides the masks of the traffic signs)
folder_crops_name = 'black_images'
# Set the path to the new folder in the Colab file system
folder_crops_path = os.path.join('/content/', folder_crops_name)
# Create the new folder
try:
  os.mkdir(folder_crops_path)
  print(f"Folder '{folder_crops_name}' created successfully at '{folder_crops_path}'")
except FileExistsError:
  print(f"Folder '{folder_crops_name}' already exists at '{folder_crops_path}'")

# folder for test_final
folder_crops_name = 'test_final'
# Set the path to the new folder in the Colab file system
folder_crops_path = os.path.join('/content/', folder_crops_name)
# Create the new folder
try:
  os.mkdir(folder_crops_path)
  print(f"Folder '{folder_crops_name}' created successfully at '{folder_crops_path}'")
except FileExistsError:
  print(f"Folder '{folder_crops_name}' already exists at '{folder_crops_path}'")

# folder for masks
folder_crops_name = 'masks'
# Set the path to the new folder in the Colab file system
folder_crops_path = os.path.join('/content/test_final/', folder_crops_name)
# Create the new folder
try:
  os.mkdir(folder_crops_path)
  print(f"Folder '{folder_crops_name}' created successfully at '{folder_crops_path}'")
except FileExistsError:
  print(f"Folder '{folder_crops_name}' already exists at '{folder_crops_path}'")

# folder for imges_png
folder_crops_name = 'images_png'
# Set the path to the new folder in the Colab file system
folder_crops_path = os.path.join('/content/', folder_crops_name)
# Create the new folder
try:
  os.mkdir(folder_crops_path)
  print(f"Folder '{folder_crops_name}' created successfully at '{folder_crops_path}'")
except FileExistsError:
  print(f"Folder '{folder_crops_name}' already exists at '{folder_crops_path}'")

# folder for output
folder_crops_name = 'output'
# Set the path to the new folder in the Colab file system
folder_crops_path = os.path.join('/content/', folder_crops_name)
# Create the new folder
try:
  os.mkdir(folder_crops_path)
  print(f"Folder '{folder_crops_name}' created successfully at '{folder_crops_path}'")
except FileExistsError:
  print(f"Folder '{folder_crops_name}' already exists at '{folder_crops_path}'")

# folder for final_images
folder_crops_name = 'final_images'
# Set the path to the new folder in the Colab file system
folder_crops_path = os.path.join('/content/', folder_crops_name)
# Create the new folder
try:
  os.mkdir(folder_crops_path)
  print(f"Folder '{folder_crops_name}' created successfully at '{folder_crops_path}'")
except FileExistsError:
  print(f"Folder '{folder_crops_name}' already exists at '{folder_crops_path}'")

Taking the video and change it back to frames

In [ ]:
!ffmpeg -i out_1000000.hevc -vf fps=30 /content/images_png/out%d.png

Sorting the frames in folder images_png

In [ ]:
import os

folder_path = '/content/images_png'  # Replace with the actual folder path

# Get the list of files in the folder
file_list = os.listdir(folder_path)

# Define a sorting key function
def get_numeric_part(filename):
    filename = filename[:-4]
    return int(filename[3::])

# Sort the file list based on the numeric part of the filename
sorted_files = sorted(file_list, key=get_numeric_part)

# Rename the files with a new order
for i, file_name in enumerate(sorted_files, start=1):
    old_path = os.path.join(folder_path, file_name)
    new_name = f'out{i:03d}.png'
    new_path = os.path.join(folder_path, new_name)
    os.rename(old_path, new_path)


Copy of the folder "images_png" (the original images)

In [ ]:
from numpy.core.fromnumeric import shape
from numpy import double
import shutil

source_folder="/content/images_png"
destination_folder="/content/images_png_copy"
shutil.copytree(source_folder, destination_folder)

Darkening the images with the masks


In [ ]:
import os
import cv2
import numpy as np
folder_path_copy = '/content/images_png_copy'
frames= os.listdir(folder_path_copy)

for frame in frames:
  frame_path_copy = os.path.join(folder_path_copy, frame)
  im_copy=cv2.imread(frame_path_copy)
  # Load the image array (im_copy) and define the range of RGB values to modify
  lower_rgb = np.array([6, 156, 118])  # Lower range of RGB values
  upper_rgb = np.array([50, 200, 162])  # Upper range of RGB values

  # Apply the modification to the image array
  im_copy[np.any(im_copy <= lower_rgb, axis=-1)]=[0, 0, 0]
  im_copy[np.any(im_copy >= upper_rgb, axis=-1)]=[0, 0, 0]
  cv2.imwrite(f'/content/black_images/{frame}', im_copy)

Finding the coordinates of the block and cut the mask

In [ ]:
import cv2
import numpy as np

folder_path_black= '/content/black_images'
frames= os.listdir(folder_path_black)

for frame in frames:
  frame_path_black = os.path.join(folder_path_black, frame)
  im_blacked =cv2.imread(frame_path_black)

  # Define the range of RGB values for the blocking square
  lower_rgb = np.array([6, 156, 118])
  upper_rgb = np.array([50, 200, 162])

  # Find the contours of the blocking square
  mask = cv2.inRange(im_blacked, lower_rgb, upper_rgb)
  contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

  # Find the bounding rectangle of the largest contour (blocking square)
  if len(contours) > 0:
      largest_contour = max(contours, key=cv2.contourArea)
      x, y, w, h = cv2.boundingRect(largest_contour)

      # Crop the blocking square from the frame
      blocking_square = im_blacked[y:y+h, x:x+w]

      cv2.imwrite(f'/content/test_final/masks/{frame}', blocking_square)
  else:
      print('No blocking square found in the frame.')

Uploading the autoencoder model

In [ ]:
from keras.models import load_model
# Load the saved model
autoencoder = load_model("/content/model_208_1.h5")

Using the model for reconstructing the traffic signs

In [ ]:
from keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Input, Concatenate
from keras.models import Model
import keras.backend as K
from keras.callbacks import EarlyStopping

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import shutil
import os

import cv2
import numpy as np

K.clear_session()

img_height = 128
img_width =  128
batch_size = 178 ###change the number according to the number of the frames

# Load test data2
test_data_generator = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
test_dataset = test_data_generator.flow_from_directory(
    directory='/content/test_final',
    target_size=(img_height, img_width),
    color_mode="rgb",
    batch_size=batch_size,
    class_mode=None,
    shuffle=False,
    seed=42
)

num_test_images = 178 ###change the number according to the number of the frames
test_images = next(test_dataset)[0:num_test_images]
reconstructed_images = autoencoder.predict(test_images)

# fig, axs = plt.subplots(nrows=num_test_images, ncols=2, figsize=(20, 30))
# for i in range(0, num_test_images):
#    axs[i, 0].imshow(test_images[i])
#    axs[i, 0].set_title("Original")
#    axs[i, 1].imshow(reconstructed_images[i])
#    axs[i, 1].set_title("Reconstructed")
# plt.tight_layout()
# plt.show()

Saving reconstructions

In [ ]:
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np
# Save reconstructed images
output_directory = '/content/reconstruction_of_traffic_signs'

# Create the output directory if it doesn't exist
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

for i in range(0, num_test_images):
  bgr_image = cv2.cvtColor(reconstructed_images[i], cv2.COLOR_RGB2BGR)
  cv2.imwrite(f'/content/reconstruction_of_traffic_signs/traffic_sign_{i}.png', bgr_image*255)

Sorting and rename images in folder reconstruction_of_traffic_signs

In [ ]:
import os

folder_path = '/content/reconstruction_of_traffic_signs'  # Replace with the actual folder path

# Get the list of files in the folder
file_list = os.listdir(folder_path)

# Define a sorting key function
def get_numeric_part(filename):
    filename = filename[:-4]
    return int(filename[13::])

# Sort the file list based on the numeric part of the filename
sorted_files = sorted(file_list, key=get_numeric_part)

# Rename the files with a new order
for i, file_name in enumerate(sorted_files, start=1):
    old_path = os.path.join(folder_path, file_name)
    new_name = f'out{i:03d}.png'
    new_path = os.path.join(folder_path, new_name)
    os.rename(old_path, new_path)

Resize reconstruction

In [ ]:
import cv2
import numpy as np
from PIL import Image

folder_original_frames="/content/images_png_copy"
folder_path_black= '/content/black_images'
folder_path_reconstruction= '/content/reconstruction_of_traffic_signs'
frames= os.listdir(folder_path_black)

for frame in frames:
  frame_path_black = os.path.join(folder_path_black, frame)
  frame_path_reconstruction = os.path.join(folder_path_reconstruction, frame)
  im_blacked =cv2.imread(frame_path_black)
  original_frame= os.path.join(folder_original_frames, frame)
  img=cv2.imread(original_frame)

  # Define the range of RGB values for the blocking square
  lower_rgb = np.array([6, 156, 118])
  upper_rgb = np.array([50, 200, 162])

  # Find the contours of the blocking square
  mask = cv2.inRange(im_blacked, lower_rgb, upper_rgb)
  contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

  # Find the bounding rectangle of the largest contour (blocking square)
  if len(contours) > 0:
      largest_contour = max(contours, key=cv2.contourArea)
      x, y, w, h = cv2.boundingRect(largest_contour)

  image = cv2.imread(frame_path_reconstruction)

  # Define the desired width and height for the resized image
  desired_width = w
  desired_height = h

  # Resize the image
  resized_image = cv2.resize(image, (desired_width, desired_height))
  #################################################################

  pil_image = Image.open(original_frame)

  # Convert PIL image to NumPy array
  original = np.array(pil_image)
  original = cv2.cvtColor(original, cv2.COLOR_RGB2BGR)
  original[y:y+h, x:x+w]=resized_image
  cv2.imwrite(f'/content/output/{frame}', original)

Putting the reconstructed traffic signs on the frames

In [ ]:
import cv2
import numpy as np

folder_source_images="/content/images_png_copy"
folder_target_images= '/content/output'

for frame in frames:
  frame_path_source = os.path.join(folder_source_images, frame)
  frame_path_target = os.path.join(folder_target_images, frame)

  # Load the source and target images
  source_image = cv2.imread(frame_path_source)
  target_image = cv2.imread(frame_path_target)

  # Create a mask for non-zero pixels in the source image
  mask = np.all(target_image == [0, 0, 0], axis=-1)

  # Replace pixels in the target image using the mask
  target_image[mask] = source_image[mask]

  # Save the modified target image
  cv2.imwrite(f"/content/final_images/{frame}", target_image)

In [ ]:
import shutil

# shutil.rmtree('/content/images_png_copy')
# shutil.rmtree('/content/extracted_folder')
# shutil.rmtree('/content/test_final')
# shutil.rmtree('/content/black_images')
# shutil.rmtree('/content/images_png')
shutil.rmtree('/content/reconstruction_of_traffic_signs')
# shutil.rmtree('/content/resize_reconstruction')
#shutil.rmtree('/content/output')
#shutil.rmtree('/content/final_images')
# shutil.rmtree('/content/kvazaar')

# Creating a video from frames

In [ ]:
!ffmpeg -framerate 30 -pattern_type glob -i "/content/final_images/*.png" output.avi
